INPUT: A polynomial in two classes of variables, $s_0 \dots s_{n_s-1}$ and $q_0 \dots q_{n_q -1}$.
The s-variables are controlled by player 2 and the q-variables are controlled by player 1.

Idea: Check the coefficent for all different monomials in s-variables that occur in the polynomial. 

If at any point we find a monomial with a coefficient that doesn't use any q-variables, we are done. Note that any term in the coefficient which has a factor of an s-variable should be ignored in this accounting, since it is part of a different monomial.

If we find such a monomial, Player 2 wins.
Else, Player 1 can force p to be the zero polynomial by choosing values for the q-variables.

In [2]:
def check_monomial_coefficient(coeff, q, s, debug=False):
    
    found_q_free_term = 0
    
    if coeff.operator() is not sage.symbolic.operators.add_vararg:
        operands = [coeff]
    else:
        operands = coeff.operands()
    
    if debug:
        print("Coefficient terms: " + str(operands))
    
    for coeff_term in operands: # each term in the coefficient sum
        
        s_free = 1
        q_free = 1
        
        for q_var in q:
            if coeff_term.degree(q_var) > 0:
                q_free = 0
                break
        for s_var in s:
            if coeff_term.degree(s_var) > 0:
                s_free = 0
                break
        
        if q_free == 0 and s_free == 1:
            # there is a true q-term in the coefficient, so P2 doesn't win with this monomial
            return False
        
        if q_free == 1:
            found_q_free_term = 1
            
    if found_q_free_term == 0:
        return False
    return True

In [3]:
# Step 1: Cycle through all monomials; that is, all combinations of the t and s variables that appear in the polynomial.
# Step 2: For each one: Check the coefficient. Ignore terms with both q's AND s/t's. If what remains has no q's, P2 wins.

def check_P2_win(p, q, s, debug=False):
    
    if type(p) is not sage.symbolic.expression.Expression:
        print("Warning: Input isn't a symbolic expression type! I'm treating it as a constant.")
        return (p != 0)
    
    if p.operator() is not sage.symbolic.operators.add_vararg:
        summands = [p]
    else:
        summands = p.expand().operands()
    if debug: 
        print("Polynomial: " + str(p))
        print("Polynomial terms: " + str(summands))
    
    for term in summands:

        if debug:
            print()
            print("Current term: " + str(term))
        
        monomial = term
        # remove all the q factors
        for q_var in q:
            if monomial.degree(q_var) > 0:
                monomial = monomial.coefficient(q_var, monomial.degree(q_var))
        if monomial.operands() and monomial.operator() is sage.symbolic.operators.mul_vararg:
            for operand in monomial.operands():
                if operand.is_constant():
                    monomial = monomial / operand

        coeff = p.coefficient(monomial)
        
        if debug:
            print("with monomial: " + str(monomial))
            print("which has coefficient: " + str(coeff))      
        
        if coeff == 0: # no s variables, just a constant coefficient in front of q's
            continue
        if check_monomial_coefficient(coeff, q, s, debug=debug):
            if debug:
                print("P2 wins with monomial " + str(monomial) + " and coefficient (" + str(coeff) +")")
            return True
        
    return False
       

In [4]:
# Set up an arbitrary instance
# s are the slack variables, q are the adversarial "constants" chosen by P1

n_s = 5
s = list(var('s%d' % i) for i in range(n_s))
n_q = 3
q = list(var('q%d' % i) for i in range(n_q))

p = s[0]*q[0] + s[0]*q[1] + s[0]*s[1]*q[2] + s[2]*s[2]*s[0]*q[1] + s[2]^2 + 1

In [7]:
# Test it out: this will return True and exhibit the steps used to determine it
check_P2_win(p, q, s, debug=True)

Polynomial: q1*s0*s2^2 + q2*s0*s1 + q0*s0 + q1*s0 + s2^2 + 1
Polynomial terms: [q1*s0*s2^2, q2*s0*s1, q0*s0, q1*s0, s2^2, 1]

Current term: q1*s0*s2^2
with monomial: s0*s2^2
which has coefficient: q1
Coefficient terms: [q1]

Current term: q2*s0*s1
with monomial: s0*s1
which has coefficient: q2
Coefficient terms: [q2]

Current term: q0*s0
with monomial: s0
which has coefficient: q1*s2^2 + q2*s1 + q0 + q1
Coefficient terms: [q1*s2^2, q2*s1, q0, q1]

Current term: q1*s0
with monomial: s0
which has coefficient: q1*s2^2 + q2*s1 + q0 + q1
Coefficient terms: [q1*s2^2, q2*s1, q0, q1]

Current term: s2^2
with monomial: s2^2
which has coefficient: q1*s0 + 1
Coefficient terms: [q1*s0, 1]
P2 wins with monomial s2^2 and coefficient (q1*s0 + 1)


True

In [10]:
# Set up polynomials used in the proof
def make_polynomials(s, q):
    return {
        "a1": s[1] + s[2],
        "a2": q[0] + s[0] + s[1] + s[2] + s[3],
        "a3": q[1] + q[2] + s[1] + s[2] + 2*s[3],
        "a4": q[0] + s[0] + s[1] + s[2] + 2*s[3],
        "a4_2": q[1] + q[2] + s[0] +s[1] + s[2] + 2*s[3],
        "pj_prime": q[1]*s[3] + s[1]*s[3] + s[3]^2,
        "sj_prime": s[1]*s[3],
        "t_prime": s[3]^2,
        "t_dblprime": s[3] + s[3]^2,
        "m1": q[1]*s[2] + s[1]*s[2] + s[2]*s[3],
        "m2": q[2]*s[1] + s[1]*s[2] + s[1]*t,
        "m3": q[1]*q[2] + q[2]*s[1] + q[1]*s[2] + s[1]*s[2] + q[1]*t +q[2]*t + s[1]*t + s[2]*t + t^2,
        "m4": s[1]*s[2],
        "m5": q[0] + s[0] + q[1]*s[2] + s[1]*s[2] + t + s[2]*t,
        "m6": q[0] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + t + s[1]*t + s[2]*t,
        "m7": q[0] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + t + q[1]*t + 2*s[1]*t + s[2]*t + t^2,
        "m8": q[1]*q[2] + s[0] + q[2]*s[1] + q[1]*s[2] + s[1]*s[2] + q[1]*t + q[2]*t + s[1]*t + s[2]*t + t^2,
        "m9": q[1]*q[2] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + q[1]*t + q[2]*t + s[1]*t + s[2]*t + t^2,
        "m10": q[1]*q[2] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + q[1]*t + q[2]*t + 2*s[1]*t + s[2]*t + t^2,
        "m11": q[1]*q[2] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + q[1]*t + q[2]*t + 2*s[1]*t + 2*s[2]*t + t^2,
        "m12": q[0] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + t + q[1]*t + q[2]*t + 2*s[1]*t + 2*s[2]*t + 2*t^2,
        "m12_2": q[1]*q[2] + s[0] + q[2]*s[1] + q[1]*s[2] + 2*s[1]*s[2] + t + q[1]*t + q[2]*t + 2*s[1]*t + 2*s[2]*t + 2*t^2
    } 



In [11]:
s_letters = ["si", "sj", "sk", "si2", "sj2", "sk2", "t"]
s = list(var(s_letters[i]) for i in range(len(s_letters)))

q_letters = ["qi", "qj", "qk", "qi2", "qj2", "qk2"]
q = list(var(q_letters[i]) for i in range(len(q_letters)))


    

In [13]:
# Print setup for producing equations -- can check that these match the ones stated in the paper

s1 = [si, sj, sk, t]
q1 = [qi, qj, qk]
polys1 = make_polynomials(s1, q1)
for key, value in polys1.items():
    print(key + ": " + str(value))
    
# sanity check that none of the individual polynomials can be forced to be identically 0
for name, p in polys1.items():
    if not check_P2_win(p, q, s, debug=False):
        print("Error: The polynomial " + name)

a1: sj + sk
a2: qi + si + sj + sk + t
a3: qj + qk + sj + sk + 2*t
a4: qi + si + sj + sk + 2*t
a4_2: qj + qk + si + sj + sk + 2*t
pj_prime: qj*t + sj*t + t^2
sj_prime: sj*t
t_prime: t^2
t_dblprime: t^2 + t
m1: qj*sk + sj*sk + sk*t
m2: qk*sj + sj*sk + sj*t
m3: qj*qk + qk*sj + qj*sk + sj*sk + qj*t + qk*t + sj*t + sk*t + t^2
m4: sj*sk
m5: qj*sk + sj*sk + sk*t + qi + si + t
m6: qk*sj + qj*sk + 2*sj*sk + sj*t + sk*t + qi + si + t
m7: qk*sj + qj*sk + 2*sj*sk + qj*t + 2*sj*t + sk*t + t^2 + qi + si + t
m8: qj*qk + qk*sj + qj*sk + sj*sk + qj*t + qk*t + sj*t + sk*t + t^2 + si
m9: qj*qk + qk*sj + qj*sk + 2*sj*sk + qj*t + qk*t + sj*t + sk*t + t^2 + si
m10: qj*qk + qk*sj + qj*sk + 2*sj*sk + qj*t + qk*t + 2*sj*t + sk*t + t^2 + si
m11: qj*qk + qk*sj + qj*sk + 2*sj*sk + qj*t + qk*t + 2*sj*t + 2*sk*t + t^2 + si
m12: qk*sj + qj*sk + 2*sj*sk + qj*t + qk*t + 2*sj*t + 2*sk*t + 2*t^2 + qi + si + t
m12_2: qj*qk + qk*sj + qj*sk + 2*sj*sk + qj*t + qk*t + 2*sj*t + 2*sk*t + 2*t^2 + si + t


In [14]:
# Finally: Check each of the pairs of equations for the conditions from the theorem
# Some of the variables within an equation may be equal; and similarly across equations. 
# Consider all combinations, with the exception of those which force all variables = 0 

import itertools    
errors = False  
counter = 0
# First we decide which of the variables for the first equation are equal
# There are 3 variables (i, j, and k) any of which may be equal
# options_1 encodes these possibilities, eg. [0,0,1] indicates i = j =/= k.
options_1 = [[0,0,0], [0,0,1], [0,1,0], [0,1,1], [0,1,2]] 
for o1 in options_1:
    # Now we have to decide which of the variables for the second equation are equal
    # They may equal each other and/or the variables from options_1
    # So we take all the options for their values: reusing variables from options_1 and/or use up to 3 new variables
    # Look at all the possible ways to select three of those (with replacement)
    options_2 = itertools.product(list(set(o1)) + [3,4,5], repeat=3)
    # For simplicity of implementation, options_2 does contain some isomorphic options, eg. [0,0,3] and [0,0,4].
    # If implementation needs to be more efficient, remove duplicates.
    
    for o2 in options_2:
        
        # Total iterations inside the nested loops: 655
        counter = counter + 1
        print(counter)
        
        # Skip ones where (j,k) in poly1 = (j,k) in poly2
        if o1[1] == o2[1] and o2[2] == o2[2]:
            continue
        if o1[2] == o2[1] and o1[1] == o2[2]:
            continue
            
        # If j = k on L or R and it's single multiplication, skip m2?
        # Skip m12-m12 and a4-a4?

        s1 = [s[o1[0]], s[o1[1]], s[o1[2]], t]
        q1 = [q[o1[0]], q[o1[1]], q[o1[2]]]
        
        s2 = [s[o2[0]], s[o2[1]], s[o2[2]], t]
        q2 = [q[o2[0]], q[o2[1]], q[o2[2]]]


        polys1 = make_polynomials(s1, q1)
        polys2 = make_polynomials(s2, q2)

        for name1, p1 in polys1.items():
            for name2, p2 in polys2.items():
                if name2 in ["t_prime", "t_dblprime"]:
                    continue # these don't have any ijk indexed variables so we don't need a second copy
                if name2 in ["pj_prime", "sj_prime"] and name1 == name2 and q1[1] == q2[1]:
                    continue # these two only have j variables -- there's only one generated per unique j so no need to compare.
                if not check_P2_win(p1 - p2, q, s, debug=False):
                    print("Error (p1 - p2): The polynomials " + name1 + " and " + name2 + " with variables " + str([s1, s2]))
                    errors = True
                if not check_P2_win(p1 + p2 - 1, q, s, debug=False):
                    print("Error (p1 + p2 - 1): The polynomials " + name1 + " and " + name2 + " with variables " + str([s1, s2]))
                    errors = True
    
print(errors)
print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
Error (p1 - p2): The polynomials a2 and a2 with variables [[si, sj, si, t], [sj, si, si, t]]
Error (p1 - p2): The polynomials a4 and a4 with variables [[si, sj, si, t], [sj, si, si, t]]
Error (p1 - p2): The polynomials a4 and a4_2 with variables [[